## Porto Seguro’s Safe Driver Prediction

Porto Seguro, one of Brazil’s largest auto and homeowner insurance companies, completely agrees. Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones.

In this competition, you’re challenged to build a model that predicts the probability that a driver will initiate an auto insurance claim in the next year. While Porto Seguro has used machine learning for the past 20 years, they’re looking to Kaggle’s machine learning community to explore new, more powerful methods. A more accurate prediction will allow them to further tailor their prices, and hopefully make auto insurance coverage more accessible to more drivers.

### Data Description

In this competition, you will predict the probability that an auto insurance policy holder files a claim.
In the train and test data, features that belong to similar groupings are tagged as such in the feature names (e.g., ind, reg, car, calc). In addition, feature names include the postfix bin to indicate binary features and cat to indicate categorical features. Features without these designations are either continuous or ordinal. Values of -1 indicate that the feature was missing from the observation. The target columns signifies whether or not a claim was filed for that policy holder.

## Starters code using Random Forest
In this notebook, we are going to use a ensemble machine learning, Random Forest model to predict wheteher a driver makes an insurance claim in the next year. 

In [1]:
### IMPORTING REQUIRED PACKAGES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# machine learning modules
import sklearn
print(sklearn.__version__)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score

0.18.1


In [2]:
#### LOADING DATA ####

### TRAIN DATA
train_data = pd.read_csv('train.csv', na_values='-1')\
                        
## Filling the missing data NAN with median of the column
train_data_nato_median = pd.DataFrame()
for column in train_data.columns:
    train_data_nato_median[column] = train_data[column].fillna(train_data[column].median())

train_data = train_data_nato_median.copy()

### TEST DATA
test_data = pd.read_csv('test.csv', na_values='-1')
## Filling the missing data NAN with mean of the column
test_data_nato_median = pd.DataFrame()
for column in test_data.columns:
    test_data_nato_median[column] = test_data[column].fillna(test_data[column].median())
    
test_data = test_data_nato_median.copy()
test_data_id = test_data.pop('id')

In [3]:
## Identifying Categorical data
column_names = train_data.columns
categorical_column = column_names[column_names.str[10] == 'c']

## Changing categorical columns to category data type
def int_to_categorical(data):
    """ 
    changing columns to catgorical data type
    """
    for column in categorical_column:
        data[column] =  data[column].astype('category')

In [4]:
## Creating list of train and test data and converting columns of interest to categorical type
datas = [train_data,test_data]

for data in datas:
    int_to_categorical(data)

test_data.dtypes

ps_ind_01            int64
ps_ind_02_cat     category
ps_ind_03            int64
ps_ind_04_cat     category
ps_ind_05_cat     category
ps_ind_06_bin        int64
ps_ind_07_bin        int64
ps_ind_08_bin        int64
ps_ind_09_bin        int64
ps_ind_10_bin        int64
ps_ind_11_bin        int64
ps_ind_12_bin        int64
ps_ind_13_bin        int64
ps_ind_14            int64
ps_ind_15            int64
ps_ind_16_bin        int64
ps_ind_17_bin        int64
ps_ind_18_bin        int64
ps_reg_01          float64
ps_reg_02          float64
ps_reg_03          float64
ps_car_01_cat     category
ps_car_02_cat     category
ps_car_03_cat     category
ps_car_04_cat     category
ps_car_05_cat     category
ps_car_06_cat     category
ps_car_07_cat     category
ps_car_08_cat     category
ps_car_09_cat     category
ps_car_10_cat     category
ps_car_11_cat     category
ps_car_11          float64
ps_car_12          float64
ps_car_13          float64
ps_car_14          float64
ps_car_15          float64
p

In [5]:
## Decribing categorical variables
# def decribe_Categorical(x):
#     """ 
#     Function to decribe Categorical data
#     """
#     from IPython.display import display, HTML
#     display(HTML(x[x.columns[x.dtypes =="category"]].describe().to_html))

# decribe_Categorical(train_data) 

In [6]:
### FUNCTION TO CREATE DUMMIES COLUMNS FOR CATEGORICAL VARIABLES
def creating_dummies(data):
    """creating dummies columns categorical varibles
    """
    for column in categorical_column:
        dummies = pd.get_dummies(data[column],prefix=column)
        data = pd.concat([data,dummies],axis =1)
        ## dropping the original columns ##
        data.drop([column],axis=1,inplace= True)

In [7]:
### CREATING DUMMIES FOR CATEGORICAL VARIABLES  
for column in categorical_column:
        dummies = pd.get_dummies(train_data[column],prefix=column)
        train_data = pd.concat([train_data,dummies],axis =1)
        train_data.drop([column],axis=1,inplace= True)


for column in categorical_column:
        dummies = pd.get_dummies(test_data[column],prefix=column)
        test_data = pd.concat([test_data,dummies],axis =1)
        test_data.drop([column],axis=1,inplace= True)

print(train_data.shape)
print(test_data.shape)

(595212, 220)
(892816, 218)


In [8]:
#Define covariates in X and dependent variable in y
X = train_data.iloc[:,2:] ## FEATURE DATA
y= train_data.target ### LABEL DATA

### CHECKING DIMENSIONS
print(X.shape)
print(y.shape)

(595212,)

In [9]:
#### SPLITTING DATA INTO TRAIN AND TEST SETS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)


### RANDOM FOREST CLASSIFIER

"""
number of estimators: 200
out of bagging set to True
N_jobs: Use all the available cores= -1
min_sample_leaf: minimum number of samples required to be at a leaf node
"""
RF_model_cat= RandomForestClassifier(200,oob_score=True,random_state=13,
                                     n_jobs = -1, min_samples_leaf = 100)

In [10]:
### FITTING RANDOM MODEL 
RF_model_cat.fit(X_train, y_train)

#Obtain class predictions
y_pred_RF_prob = RF_model_cat.predict_proba(X_test)
print('Predicted probabilities: \n', y_pred_RF_prob)

#Obtain probability predictions
y_pred_RF_class = RF_model_cat.predict(X_test)
print('Predicted classes: \n', y_pred_RF_class)

print('RF Score: ', metrics.accuracy_score(y_test, y_pred_RF_class))

## CONFUSION MATRIX
RF_cm=metrics.confusion_matrix(y_test,y_pred_RF_class)
print(RF_cm)

Predicted probabilities: 
 [[ 0.97157923  0.02842077]
 [ 0.96437336  0.03562664]
 [ 0.97800145  0.02199855]
 ..., 
 [ 0.97366796  0.02633204]
 [ 0.97002314  0.02997686]
 [ 0.96523639  0.03476361]]
Predicted classes: 
 [0 0 0 ..., 0 0 0]
RF Score:  0.963374394333
[[143353      0]
 [  5450      0]]


In [11]:
#### Predicition on test data ####
y_pred_RF_prob = RF_model_cat.predict_proba(test_data)
pred_values= pd.DataFrame(y_pred_RF_prob)

submission_simple_RF= pd.DataFrame()
submission_simple_RF['id'] = test_data_id

submission_simple_RF['target'] = pd.DataFrame(pred_values.iloc[:,1])
submission_simple_RF = submission_simple_RF.set_index('id')

submission_simple_RF.columns
submission_simple_RF.head()
## Write to CSV
submission_simple_RF.to_csv("Simple Random Forest.csv")